<a href="https://colab.research.google.com/github/2796gaurav/code_examples/blob/main/Qlib_india/Qlib_indian_stocks_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Indian stocks data into Qlib

* Detailed information is in Qlib readme: https://github.com/microsoft/qlib/tree/main/scripts/data_collector/yahoo

In [1]:
################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

! pip install pyqlib yahooquery

In [ ]:
# repo name changed to qlib_repo
!git clone https://github.com/microsoft/qlib.git qlib_repo

## Collector YahooFinance data to qlib
### collector YahooFinance data and dump into qlib format

download data to csv: `python scripts/data_collector/yahoo/collector.py download_data`

parameters:
* source_dir: save the directory
* interval: 1d or 1min, by default 1d
due to the limitation of the YahooFinance API, only the last month's data is available in 1min

* region: CN or US or IN, by default CN
* delay: time.sleep(delay), by default 0.5
* start: start datetime, by default "2000-01-01"; closed interval(including start)
* end: end datetime, by default pd.Timestamp(datetime.datetime.now() + pd.Timedelta(days=1)); open interval(excluding end)
* max_workers: get the number of concurrent symbols, it is not recommended to change this parameter in order to maintain the integrity of the symbol data, by default 1
* check_data_length: check the number of rows per symbol, by default None
if len(symbol_df) < check_data_length, it will be re-fetched, with the number of re-fetches coming from the max_collector_count parameter

* max_collector_count: number of "failed" symbol retries, by default 2

In [ ]:
# Get data from YahooQuery in CSV format

!python qlib_repo/scripts/data_collector/yahoo/collector.py download_data --source_dir ~/.qlib/stock_data/source/in_1d --start 2020-01-01 --end 2020-12-31 --delay 1 --interval 1d --region IN

## Normalize Yahooquery data

normalize data: `python scripts/data_collector/yahoo/collector.py normalize_data`

parameters:
* source_dir: csv directory
* normalize_dir: result directory
* max_workers: number of concurrent, by default 1
* interval: 1d or 1min, by default 1d
if interval == 1min, qlib_data_1d_dir cannot be None

* region: CN or US or IN, by default CN
* date_field_name: column name identifying time in csv files, by default date
* symbol_field_name: column name identifying symbol in csv files, by default symbol
* end_date: if not None, normalize the last date saved (including end_date); if None, it will ignore this parameter; by default None


In [3]:
!python qlib_repo/scripts/data_collector/yahoo/collector.py normalize_data --source_dir ~/.qlib/stock_data/source/in_1d --normalize_dir ~/.qlib/stock_data/source/in_1d_nor --region IN --interval 1d

2021-08-26 14:52:02.347 | INFO     | data_collector.utils:get_calendar_list:64 - get calendar list: IN_ALL......
2021-08-26 14:52:02.504 | INFO     | data_collector.utils:get_calendar_list:100 - end of get calendar list: IN_ALL.
2021-08-26 14:52:02.504 | INFO     | data_collector.base:normalize:301 - normalize data......
100% 25/25 [00:01<00:00, 18.49it/s]


## Convert normalized data to .bin format

dump data: `python scripts/dump_bin.py dump_all`

parameters:
* csv_path: stock data path or directory, normalize result(normalize_dir)
* qlib_dir: qlib(dump) data director
* freq: transaction frequency, by default day
* freq_map = {1d:day, 1mih: 1min}

* max_workers: number of threads, by default 16
* include_fields: dump fields, by default ""
* exclude_fields: fields not dumped, by default `"""
* dump_fields = include_fields if include_fields else set(symbol_df.columns) - set(exclude_fields) exclude_fields else symbol_df.columns

* symbol_field_name: column name identifying symbol in csv files, by default symbol
* date_field_name: column name identifying time in csv files, by default date

In [4]:
!python qlib_repo/scripts/dump_bin.py dump_all --csv_path ~/.qlib/stock_data/source/in_1d_nor --qlib_dir ~/.qlib/qlib_data/qlib_in_1d --freq day --exclude_fields date,symbol

2021-08-26 14:52:05.177 | INFO     | __main__:_get_all_date:266 - start get all date......
100% 25/25 [00:00<00:00, 64.04it/s]
2021-08-26 14:52:05.570 | INFO     | __main__:_get_all_date:285 - end of get all date.

2021-08-26 14:52:05.571 | INFO     | __main__:_dump_calendars:288 - start dump calendars......
2021-08-26 14:52:05.574 | INFO     | __main__:_dump_calendars:291 - end of calendars dump.

2021-08-26 14:52:05.574 | INFO     | __main__:_dump_instruments:294 - start dump instruments......
2021-08-26 14:52:05.575 | INFO     | __main__:_dump_instruments:296 - end of instruments dump.

2021-08-26 14:52:05.575 | INFO     | __main__:_dump_features:299 - start dump features......
100% 25/25 [00:00<00:00, 42.62it/s]
2021-08-26 14:52:06.162 | INFO     | __main__:_dump_features:306 - end of features dump.

